In [ ]:
import os
import json
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableLambda
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from dotenv import load_dotenv
from crewai.tools import tool

# Load environment variables (ensure OPENAI_API_KEY is set)
load_dotenv()
# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)

# Define document directory
DOCUMENTS_DIR = "../outputs/"
DOCUMENTS = [
    "1.rfp_comparative_analysis.md",
    "2.pricing_risk_analysis.md",
    "3.negotiation_charter.md",
    "4.negotiation_email.md",
    "5.counter_offer_email.md",
    "5a.counteroffer_strategy.md",
    "6.final_contract.md",  # The final contract to be reviewed
]

def load_documents():
    """Reads and combines all negotiation documents and the final contract."""
    context_data = ""
    contract_text = ""

    for doc in DOCUMENTS:
        file_path = os.path.join(DOCUMENTS_DIR, doc)
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                if "final_contract" in doc:
                    contract_text = content  # Identify the final contract separately
                else:
                    context_data += f"\n### {doc}\n" + content + "\n\n"
        else:
            print(f"⚠️ Warning: {doc} not found!")

    if not contract_text:
        raise ValueError("❌ Error: Final contract document not found in ./outputs/!")

    return contract_text, context_data

# Define structured output schema
response_schemas = [
    ResponseSchema(name="key_deviations", description="List of key deviations and discrepancies found."),
    ResponseSchema(name="recommended_corrections", description="Detailed correction recommendations."),
    ResponseSchema(name="final_verdict", description="Overall review conclusion: Acceptable, Minor Fixes, or Major Revisions."),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# Define prompt for LLM
prompt_template = PromptTemplate(
    input_variables=["contract", "context"],
    template="""
    You are an **AI-powered legal contract reviewer** specializing in procurement and supplier negotiations.
    
    Your task is to review the **final contract** and compare it against the provided **negotiation documents**.
    Highlight any **deviations, missing clauses, risks, or non-compliance issues**.

    ---
    ## **📌 Review Criteria:**
    - **Pricing & Payment Terms:** Ensure alignment with negotiated pricing & risk mitigation strategies.  
    - **Service Level Agreements (SLAs):** Validate penalties, uptime guarantees, and performance requirements.  
    - **Negotiated Benefits:** Ensure agreed-upon **discounts, incentives, pricing flexibility** are included.  
    - **Regulatory Compliance:** Verify **GDPR, SOC2, ISO27001, and other obligations** are present.  
    - **Dispute Resolution:** Assess arbitration clauses for fairness & clarity.  
    - **Exit Clauses & Contract Flexibility:** Confirm renegotiation, termination, and renewal terms exist.  

    ---
    ## **📜 Reference Negotiation Documents:**
    ```markdown
    {context}
    ```

    ---
    ## **📝 Final Contract:**
    ```markdown
    {contract}
    ```

    ---
    {format_instructions}
    """,
    partial_variables={"format_instructions": format_instructions}
)

# Create a chain
chain = prompt_template | llm | output_parser


def review_contract():
    """Runs the contract review process using LLM."""
    contract_text, context = load_documents()
    review = chain.invoke({"contract": contract_text, "context": context})
    return review

In [ ]:

# ✅ **Run the review tool**
contract_review_output = review_contract()

In [ ]:
contract_review_output

In [ ]:
print(contract_review_output)

In [ ]:
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# ✅ Ensure the function is a CrewAI-compatible tool
@tool
def review_contract_tool(contract_text: str, reference_documents: dict) -> str:
    """
    Sends the final contract along with reference negotiation documents to an LLM for a detailed review.
    The LLM will analyze deviations, missing clauses, and non-compliance based on all provided documents.
    Returns structured feedback with recommended corrections.
    """
    
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    
    context = "\n\n".join([f"## {key}\n{value}" for key, value in reference_documents.items()])
    
    prompt_template = PromptTemplate(
        input_variables=["contract", "context"],
        template="""
        You are a legal expert specializing in contract negotiations. Your task is to review the final contract against
        the provided negotiation documents and highlight any deviations, missing clauses, or non-compliance issues.
        
        Consider the following factors:
        1. **Pricing & Payment Terms:** Ensure the contract aligns with pricing agreements and risk mitigation strategies.
        2. **Service Level Agreements (SLAs):** Validate that penalties and uptime guarantees match the negotiation.
        3. **Negotiated Benefits:** Check for the inclusion of agreed-upon discounts, incentives, and pricing flexibility.
        4. **Regulatory Compliance:** Verify compliance with GDPR, SOC2, ISO27001, and other applicable laws.
        5. **Dispute Resolution:** Ensure arbitration or conflict resolution terms are well defined.
        6. **Exit Clauses & Contract Flexibility:** Confirm the presence of renegotiation, termination, and renewal terms.
        
        Provide a detailed review with necessary corrections and recommendations in markdown format.
        
        ## Reference Documents:
        {context}
        
        ## Final Contract:
        {contract}
        
        ---
        
        ## Contract Review Report
        ### 1. **Key Deviations Identified**
        - [ ] Pricing & Payment Terms: (Highlight discrepancies)
        - [ ] SLAs & Penalty Clauses: (Check compliance)
        - [ ] Negotiated Benefits: (Ensure supplier commitments are reflected)
        - [ ] Regulatory Compliance: (Verify adherence to legal standards)
        - [ ] Dispute Resolution: (Assess clarity and fairness)
        - [ ] Exit Clauses: (Confirm flexibility and buyer protection)
        
        ### 2. **Recommended Corrections**
        - **Clause X:** (Specify correction needed)
        - **Clause Y:** (Recommend additional safeguards)
        - **General:** (List any missing provisions or clarifications)
        
        ### 3. **Final Verdict**
        - [ ] Acceptable as-is
        - [ ] Requires minor modifications
        - [ ] Major revisions needed before approval
        
        ---
        
        Provide a structured analysis with actionable insights for legal and procurement teams.
        """
    )
    
    chain = LLMChain(llm=llm, prompt=prompt_template)
    review_feedback = chain.invoke({"contract": contract_text, "context": context})
    
    return review_feedback


: 